In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import train_flow
import importlib
importlib.reload(train_flow)
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Feature Engineering for net new features

In [406]:
# --------- #
# LEAVE ME  #
# --------- #
ticker = 'QQQ'
include_minute_feats = "N"
returns = [1, 2, 3, 5, 10, 20, 30]
df_daily, feature_sets, return_cols, daily_cols, feature_dict, features = train_flow.import_data(ticker, include_minute_feats, returns)

# ---------- #
# UPDATE ME  #
# ---------- #
# Add any new features
"""
df_daily[[f"{c}_sum10" for c in df_daily.columns if c.startswith("Past_Return_")]] = (df_daily.sort_values(by="Date", ascending=True).filter(like="Past_Return_").rolling(10, min_periods=1).sum())
past_ret_cols = [c for c in df_daily.columns if c.startswith("Past_Return%") or c.endswith("sum10")]
past_perc_cols = [c for c in df_daily.columns if c.startswith("Past_Return%")]
past_sum_cols = [c for c in df_daily.columns if c.endswith("sum10")]
#cutoff_date = '2026-02-06' # Unlessretraining all baseline models, leave alone
"""
# --------- #
# LEAVE ME  #
# --------- #
df_main = df_daily.copy()#[df_daily['Date'] <= cutoff_date].copy() #cutoff date for baseline performance metrics

Available Feature Sets: dict_keys(['ma', 'rsi', 'macd', 'volume', 'atr_adx', 'volatility', 'vix_skew', 'experimental_slope', 'past_return'])


# Run Train_Flow

In [ ]:
importlib.reload(train_flow)
from itertools import combinations

# --------- #
# LEAVE ME  #
# --------- #
# Handling of new features and permutation importance
pi_handlings = ['run_separately'] #'run_together', 'run_separately'
train_flow.pi_handling_test(pi_handlings)
keys = list(feature_dict.keys())
two_combos = list(combinations(keys, 2))

master_results = []
master_perm = []


for a, b in two_combos:
# ---------- #
# UPDATE ME  #
# ---------- #
    target_horizons = [1, 2, 5, 10, 20, 30]
    train_years = [3]
    pi_years = [1.5]
    min_feats = [4]
    days_assessed = 250
    test_day = 5
    feature_cols = feature_dict[a] + feature_dict[b]
    list_name = f"{a}-{b}"
    print(list_name)
    groups = list_name.split("-")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    models = {"xgboost-tr3-d6-c1-lr:.1": XGBClassifier(n_estimators=300, max_depth=6, colsample_bytree=1, min_child_weight=5, 
                                                        learning_rate=.1, random_state=42, n_jobs=-1)}
    models = {"xgboost-3": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, feature_dict, groups)

    master_results.append(results_df)
    #master_perm.append(perm_df)


"""
for a in keys:

    feature_cols = feature_dict[a] + feature_dict[b]
    list_name = f"{a}"
    print(list_name)
    groups = list_name.split("-")


    # --------- #
    # LEAVE ME  #
    # --------- # 
    #models = {"xgboost-3-f": XGBClassifier(n_estimators=300, random_state=42, n_jobs=-1)}
    models = {"xgboost-1-10": XGBClassifier(n_estimators=100, max_depth=10, colsample_bytree=0.75, min_child_weight=10, random_state=42, n_jobs=-1)}
    models = {"xgboost-tr3-d6-c1-lr:.1": XGBClassifier(n_estimators=300, max_depth=6, colsample_bytree=1, min_child_weight=5, 
                                                        learning_rate=.1, random_state=42, n_jobs=-1)}
    models = {"xgboost-3": XGBClassifier(n_estimators=300, min_child_weight=5, random_state=42, n_jobs=-1)}
    results_df = train_flow.run_train_flow(test_day, days_assessed, target_horizons, pi_handlings, feature_cols, df_main, models,
                    train_years, pi_years, min_feats, list_name, feature_dict, groups)

    master_results.append(results_df)
"""

# final master DataFrames
master_results_df = pd.concat(master_results, ignore_index=True)
#master_results_df.to_csv('h5_performance.csv', index=False)

ma_lag-volu
Running for horizon 5 | run_separately
Run 1/50 | Train: 2023-02-22 → 2026-01-27 | Test: 2026-02-04 → 2026-02-10 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 5)
Run 2/50 | Train: 2023-02-14 → 2026-01-20 | Test: 2026-01-28 → 2026-02-03 | Train_n=735 | Test_n=5 | (PI Years: 1.5 - Feats: 5)


KeyboardInterrupt: 

# Save most recent run

In [ ]:
rs = [1, 2, 5, 10, 20, 30]
for r in target_horizons:
    master_results_df[(master_results_df['test_days'] == 5) & (master_results_df['horizon'] == r)].to_csv(f"h{r}_performance_all.csv", index=False)

In [380]:
master_results_df = pd.concat(master_results, ignore_index=True)
base_df = pd.read_csv("h2_performance.csv")
cols = master_results_df.columns
df_concat = pd.concat([base_df.drop_duplicates(), master_results_df[cols][master_results_df['horizon'] == 2].drop_duplicates()], ignore_index=True)
df_concat.to_csv("h2_performance_all.csv", index=False)

# Run Performance_Flow

In [ ]:
import performance_flow
importlib.reload(performance_flow)

# ---------- #
# UPDATE ME  #
# ---------- #
results_file_name = "horizon_30_baseline.csv" # Match prior cell saved as file name horizon_2_baseline_new
min_th = 0.55
cov_th = 0.75


# --------- #
# LEAVE ME  #
# --------- #
return_cols, perf_df = performance_flow.import_data(results_file_name, df_daily)
composite_score = performance_flow.run_performance(perf_df, min_th, cov_th)
bucket_df = performance_flow.bucket_scores(df_daily, perf_df, returns, min_th)
cols = ['horizon', 'model', 'train_years', 'feature_set', 'pi_size',
       'min_feats', 'test_days', 'pos_rate', 'bal_prec', 'pprec', 'nprec', 'records', 'pos_rate']

composite_score[cols].head(6)

,horizon,model,train_years,feature_set,pi_size,min_feats,pos_rate,records,bal_prec,bal_prec_|0.75|,brier,log_loss,mcc,pprec,nprec,cov_|0.75|,pprec_|0.75|,nprec_|0.75|,composite
82,30,xgboost-3,5,ma_sma-experimental_slope,1.5-r,4,0.81,117.0,0.72,0.73,0.20,3.29,0.54,0.99,0.45,0.95,0.98,0.47,0.67
69,30,xgboost-3,5,ma_lag-experimental_slope,1.5-r,4,0.83,116.0,0.71,0.69,0.19,4.13,0.53,0.97,0.45,0.94,0.97,0.42,0.65
63,30,xgboost-3,5,ma_lag-ma_sma,1.5-r,4,0.83,116.0,0.70,0.71,0.18,4.63,0.51,0.96,0.45,0.96,0.96,0.46,0.65
81,30,xgboost-3,5,ma_sma-vix_skew,1.5-r,4,0.82,113.0,0.70,0.72,0.19,1.98,0.50,0.97,0.42,0.89,0.97,0.47,0.64
100,30,xgboost-3,5,ma_sma-pr,1.5-r,4,0.81,116.0,0.69,0.72,0.21,3.34,0.47,0.96,0.42,0.91,0.97,0.46,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,30,xgboost-3,5,ma_num-rsi_macd,1.5-r,4,0.82,120.0,NaN,NaN,0.18,5.01,NaN,0.82,NaN,0.99,0.82,NaN,NaN
77,30,xgboost-3,5,ma_sma-ma_num,1.5-r,4,0.82,120.0,0.58,NaN,0.17,4.17,0.06,0.82,0.33,0.97,0.82,NaN,NaN
101,30,xgboost-3,5,ma_num-pr,1.5-r,4,0.82,119.0,0.41,NaN,0.16,3.66,-0.04,0.81,0.00,0.97,0.83,NaN,NaN
84,30,xgboost-3,5,ma_num-volu,1.5-r,4,0.82,118.0,0.41,NaN,0.17,3.46,-0.04,0.82,0.00,0.97,0.82,NaN,NaN
